In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import pandas as pd

In [2]:
df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Model Building (BERT)

In [3]:
preprocessor = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/1")
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-12-h-128-a-2/versions/2", trainable=True)

## Data Preprocessing and cleaning

In [4]:
df['spam'] = df.Category.apply(lambda x: 0 if x=='ham' else 1)

In [5]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [6]:
df.Category.value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

There is an imbalance in our dataset, lets handle it using SMOTE

But first, we must do the word embedding.

In [7]:
def sentence_embedding(sentences):
  preprocessed_text = preprocessor(sentences)
  output = encoder(preprocessed_text)

  return output['pooled_output']

Now, we can use the `pooled_output` as the **feature (X)** and the `spam` as the **label (y)**

In [8]:
X = sentence_embedding(df.Message)
y = df.spam

Now, we can apply SMOTE to balance the data

In [9]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

In [10]:
y_sm.value_counts()

spam
0    4825
1    4825
Name: count, dtype: int64

## Data Splitting into train and test set

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=.2, random_state=42)

## Model Building

In [12]:
X_train.shape

(7720, 128)

In [13]:
y_train.shape

(7720,)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(128,), activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [15]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
242/242 [==============================] - 3s 4ms/step - loss: 0.3936 - accuracy: 0.8447
Epoch 2/10
242/242 [==============================] - 1s 3ms/step - loss: 0.2118 - accuracy: 0.9297
Epoch 3/10
242/242 [==============================] - 1s 4ms/step - loss: 0.1699 - accuracy: 0.9392
Epoch 4/10
242/242 [==============================] - 1s 2ms/step - loss: 0.1531 - accuracy: 0.9449
Epoch 5/10
242/242 [==============================] - 1s 2ms/step - loss: 0.1423 - accuracy: 0.9503
Epoch 6/10
242/242 [==============================] - 1s 2ms/step - loss: 0.1376 - accuracy: 0.9501
Epoch 7/10
242/242 [==============================] - 1s 2ms/step - loss: 0.1303 - accuracy: 0.9534
Epoch 8/10
242/242 [==============================] - 1s 2ms/step - loss: 0.1294 - accuracy: 0.9530
Epoch 9/10
242/242 [==============================] - 1s 2ms/step - loss: 0.1257 - accuracy: 0.9535
Epoch 10/10
242/242 [==============================] - 1s 3ms/step - loss: 0.1244 - accuracy: 0.9565

In [16]:
model.evaluate(X_test, y_test)

61/61 [==============================] - 1s 4ms/step - loss: 0.1056 - accuracy: 0.9653


[0.10560941696166992, 0.9652850031852722]

In [17]:
def predict_spam(sentence):
  preprocessed_text = preprocessor([sentence])
  output = encoder(preprocessed_text)

  spam_prob = model.predict(output['pooled_output'])

  return spam_prob

In [18]:
predict_spam('''
  OMG, DOWNLOAD FREE ROBUX USING THIS LINK BELOW
  WWW.FREE-ROBUX.COM
''')

1/1 [==============================] - 0s 155ms/step


array([[0.93381685]], dtype=float32)

In [19]:
model.save("./model.h5", save_format='h5')